In [3]:
import faiss
import pickle

Usage: author recommendation

In [7]:
# load author mapping
with open('data/idx2author.pkl', 'rb') as f:
    idx2author = pickle.load(f)

with open('data/author2idx.pkl', 'rb') as f:
    author2idx = pickle.load(f)

# load embeddings
with open('embeddings/authors.pkl', 'rb') as f:
    author_embedding = pickle.load(f)

In [5]:
# load index
index = faiss.read_index('index/author.index')

In [13]:
author_name = 'Jan Aelterman'
topn = 5
index.nprobe = 10 # number of clusters to search
try: 
    idx = author2idx[author_name]
    embedding = author_embedding[idx].unsqueeze(0).numpy()
    faiss.normalize_L2(embedding)
    _, I = index.search(embedding, topn)
except KeyError:
    print('Author not in index')

for rec in I[0][1:]: # as first is always the same author
    print(idx2author[rec])

Joris Roels
Jonas De Vylder
gerda claeskens
Emile Kelkboom


Usage: paper recommendation

In [14]:
# load embeddings
with open('embeddings/papers.pkl', 'rb') as f:
    paper_embeddings = pickle.load(f)

In [15]:
# load index
index = faiss.read_index('index/paper.index')

In [21]:
paper_id = '53e997ccb7602d9701fbe45d'
topn = 5
index.nprobe = 10 # number of clusters to search

embedding = paper_embeddings[paper_id].unsqueeze(0).numpy()
faiss.normalize_L2(embedding)
_, I = index.search(embedding, topn)


res_id = []
for rec in I[0]: # as first is always the same author
    id = list(paper_embeddings.keys())[rec]
    res_id.append(id)
    print(id)

53e997ccb7602d9701fbe45d
53e9a15bb7602d9702a4b00d
53e9a440b7602d9702d5d65b
53e9bc61b7602d97048dbf93
56d91fa3dabfae2eee95c38a


In [36]:
import pandas as pd

data = pd.read_csv('../clustering/results/004_of_V13_topics.csv')
# first row - the paper for which we searched
print(data['abstract'][data['_id'].isin(res_id)])


6        Different geometries of nitromethane dimer and...
8348     Rate coefficients for the reaction of the hydr...
10790    Vibrational analysis of tellurium tetrachlorid...
32736    The aromatic oligoamide (arylamide) foldamer c...
45479    The hydration behavior of two model disacchari...
Name: abstract, dtype: object
